In [0]:
%sql
-- Find orders whom placed orders above the average order amount
-- using sub query

select * from orders where total_amount > 
(
  select avg(total_amount) from orders 
)
order by total_amount asc

In [0]:
%sql
-- Find customers whom placed orders above the average order amount
-- using cte

with avg_total_amount as (
  select avg(total_amount) as avg_amount  from orders
)

select * from orders where total_amount > 
(select avg_amount from avg_total_amount) 
order by total_amount asc

In [0]:
%sql
-- Find Customers whom placed orders above the average order amount
-- using join and sub query

select c.first_name, c.city, o.order_date, o.total_amount  
from customers c
inner join orders o
ON c.customer_id = o.customer_id
where o.total_amount > 
(select avg(total_amount) from orders)
order by o.total_amount asc

In [0]:
%sql
-- list all the products that have been sold more than 2 times in total

select od.category as Category_Name, count(od.category) as Category_Count
from order_details od
group by od.category
having count(od.category) > 2
order by od.category asc

In [0]:
%sql
-- list customers and their total quantity of the products ordered
-- using join and cte

with customer_orders as 
(
select c.customer_id, c.first_name, sum(od.quantity) as total_quantity 
from customers c
inner join orders o
  ON c.customer_id = o.customer_id
inner join order_details od
  ON o.order_id = od.order_id
group by c.customer_id, c.first_name
having total_quantity > 3
order by c.customer_id asc
)
select * from customer_orders --where total_quantity > 3

In [0]:
%sql
-- find products with their total sales revenue (price * quantity)
with total_sales as (
select od.category, sum(od.quantity * od.price_each) as total_sales
from order_details od
group by od.category
order by total_sales asc
)
select * from total_sales where total_sales > 2000

**_CASE STATEMENT (SQL- IF/ELSE)_**

In [0]:
%sql
--categorize each order as 'small', 'median' and 'large'
-- using cte and case statement

with order_size_details as 
(
  select o.order_id, o.total_amount,
      case
        when o.total_amount > 2500 then 'large'
        when o.total_amount between 1500 and 2500 then 'median'
        else 'small'
      end as order_size
  from orders o
  order by total_amount asc
)
select * from order_size_details where order_size = 'large'

In [0]:
%sql
-- Label customers as new or returning based on order count
-- using cte and join

with customer_type_details (
  select c.customer_id, count(o.order_id) as total_orders,
      case
          when count(o.order_id) = 1 then 'new'
          when count(o.order_id) > 1 then 'retruning'
          else 'no orders'
      end as customer_type
  from customers c
  left join orders o
  on c.customer_id = o.customer_id
  group by c.customer_id
  order by c.customer_id asc, total_orders asc)

select * from customer_type_details where customer_type = 'new'

**_FUNCTIONS (STRING + DATE)_**

In [0]:
%sql
select upper(first_name) as FIRST_NAME, upper(last_name) as LAST_NAME, concat(first_name, ' ', last_name) from customers  

In [0]:
%sql
-- Extract each month from order_date column and group month and sum total_amount

with order_by_month as (
    select 
        month(order_date) as Month_Order, 
        sum(total_amount) as Total_Amount, 
        count(order_id) as Order_Count
    from orders 
  group by Month_Order
  order by Month_Order asc
)

select * from order_by_month where order_count > 5

In [0]:
%sql
--list each customer's name + their total spend + spending category

with categoty_spend (
select 
    concat_ws(' ', c.first_name, c.last_name) as full_name, 
    od.category, 
    sum(o.total_amount) as total_spend
from customers c
inner join orders o
    on c.customer_id = o.customer_id
inner join order_details od
    on o.order_id = od.order_id
group by c.first_name, c.last_name, od.category
order by full_name)

select * from categoty_spend where total_spend > 1500

In [0]:
%sql
--list each customer's name + their total spend + spending category

with categoty_spend (
select 
    concat_ws(' ', c.first_name, c.last_name) as full_name, 
    c.customer_id, 
    sum(od.quantity * od.price_each) as total_spend
from customers c
inner join orders o
    on c.customer_id = o.customer_id
inner join order_details od
    on o.order_id = od.order_id
group by c.customer_id, c.first_name, c.last_name
order by full_name)

select customer_id, full_name, total_spend,
case
  when total_spend > 20000 then 'Gold'
  when total_spend between 5000 and 20000 then 'Silver'
  else 'Bronze'
end as customer_tier
from categoty_spend;

### # 1. String Functions (T-SQL)

| Function | Syntax | Description |
|----------|--------|-------------|
| LOWER() | LOWER(string) | Converts all characters to lowercase. |
| UPPER() | UPPER(string) | Converts all characters to uppercase. |
| LEN() | LEN(string) | Returns number of characters (excluding trailing spaces). |
| LTRIM() | LTRIM(string) | Removes leading spaces. |
| RTRIM() | RTRIM(string) | Removes trailing spaces. |
| TRIM() | TRIM([characters FROM] string) | Removes leading & trailing spaces or specified characters (SQL Server 2017+). |
| SUBSTRING() | SUBSTRING(string, start, length) | Extracts part of a string. |
| REPLACE() | REPLACE(string, search, replace) | Replaces all occurrences of a substring. |
| CONCAT() | CONCAT(str1, str2, ...) | Joins strings together. |
| LEFT() | LEFT(string, n) | Returns leftmost n characters. |
| RIGHT() | RIGHT(string, n) | Returns rightmost n characters. |
| CHARINDEX() | CHARINDEX(substring, string [, start_location]) | Finds starting position of substring. |
| PATINDEX() | PATINDEX('%pattern%', string) | Finds starting position of a pattern (supports wildcards). |
| REVERSE() | REVERSE(string) | Reverses the string. |
| REPLICATE() | REPLICATE(string, count) | Repeats a string multiple times. |
| SPACE() | SPACE(number) | Returns a string of spaces. |
| STRING_AGG() | STRING_AGG(column, separator) | Concatenates values from multiple rows with a separator (SQL Server 2017+). |
| FORMAT() | FORMAT(value, format [, culture]) | Formats date, number, or string using a format pattern. |
### 2. Date & Time Functions (T-SQL)

| Function | Syntax | Description |
|----------|--------|-------------|
| GETDATE() | GETDATE() | Current date and time. |
| SYSDATETIME() | SYSDATETIME() | Current date and time with higher precision. |
| CURRENT_TIMESTAMP | CURRENT_TIMESTAMP | ANSI equivalent of GETDATE(). |
| GETUTCDATE() | GETUTCDATE() | Current UTC date and time. |
| DATEADD() | DATEADD(datepart, number, date) | Adds an interval to a date. |
| DATEDIFF() | DATEDIFF(datepart, startdate, enddate) | Difference between two dates in specified unit. |
| DATENAME() | DATENAME(datepart, date) | Returns the name of a part of a date (e.g., "Monday"). |
| DATEPART() | DATEPART(datepart, date) | Returns integer value of part of a date (e.g., month number). |
| EOMONTH() | EOMONTH(start_date [, month_to_add]) | Last day of month. |
| ISDATE() | ISDATE(expression) | Checks if expression is a valid date. |
| SWITCHOFFSET() | SWITCHOFFSET(datetimeoffset_value, time_zone_offset) | Changes time zone offset of a datetimeoffset value. |
| TODATETIMEOFFSET() | TODATETIMEOFFSET(datetime2_value, time_zone_offset) | Adds time zone offset to a datetime2 value. |
| FORMAT() | FORMAT(date, format [, culture]) | Formats date as a string. |
### 3. Numeric Functions (T-SQL)

| Function | Syntax | Description |
|----------|--------|-------------|
| ABS() | ABS(number) | Absolute value. |
| ROUND() | ROUND(number, decimals [, operation]) | Rounds to given decimals (operation=0 normal, 1 truncates). |
| CEILING() | CEILING(number) | Rounds up to nearest integer. |
| FLOOR() | FLOOR(number) | Rounds down to nearest integer. |
| POWER() | POWER(number, exponent) | Number raised to a power. |
| SQRT() | SQRT(number) | Square root. |
| RAND() | RAND([seed]) | Random number between 0–1. |
| SIGN() | SIGN(number) | -1, 0, or 1 for negative, zero, or positive numbers. |
| EXP() | EXP(number) | e raised to the given power. |
| LOG() | LOG(number [, base]) | Logarithm (natural or custom base). |
| LOG10() | LOG10(number) | Base-10 logarithm. |
| PI() | PI() | Returns π value. |
| SQUARE() | SQUARE(number) | Returns square of number. |
### 4. Aggregate Functions (T-SQL)

| Function | Syntax | Description |
|----------|--------|-------------|
| COUNT() | COUNT(column_or_*) | Row count (non-NULL if column given). |
| SUM() | SUM(column) | Total sum. |
| AVG() | AVG(column) | Average value. |
| MIN() | MIN(column) | Minimum value. |
| MAX() | MAX(column) | Maximum value. |
| STRING_AGG() | STRING_AGG(column, separator) | Concatenates values from rows (SQL Server 2017+). |
| VAR() / VARP() | VAR(column) / VARP(column) | Variance (sample / population). |
| STDEV() / STDEVP() | STDEV(column) / STDEVP(column) | Standard deviation (sample / population). |
### 5. Conditional Functions (T-SQL)

| Function | Syntax | Description |
|----------|--------|-------------|
| CASE | CASE WHEN condition THEN value ELSE value END | IF-ELSE logic. |
| COALESCE() | COALESCE(expr1, expr2, ...) | First non-NULL value. |
| NULLIF() | NULLIF(expr1, expr2) | NULL if equal, else expr1. |
| IIF() | IIF(condition, value_if_true, value_if_false) | Shorter form of CASE (SQL Server 2012+). |
